In [224]:
# prompt: load files from drive

from google.colab import drive
drive.mount('/content/drive')
#change weather folder to correct folder for the state
%cd /content/drive/MyDrive/2_WEATHER
!ls

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/2_WEATHER
CO_out.csv  CO_wwa_201401010000_202212310000.csv  SD_wwa_201401010000_202212310000.csv	test.csv


In [225]:
import pandas as pd
#change file name for each state
df = pd.read_csv('SD_wwa_201401010000_202212310000.csv',usecols=['ISSUED', 'EXPIRED', 'SIG', 'PHENOM'])
df=df.loc[df['SIG'] == 'W']
#change PHENOM for each Pheno. Comment out below line if finding total corelation
#df=df.loc[df['PHENOM'] == 'EH']
#df=df.loc[df['PHENOM'] == 'SV']
#df=df.loc[df['PHENOM'] == 'FF']
#df=df.loc[df['PHENOM'] == 'TO']
#df=df.loc[df['PHENOM'] == 'FW']
#df=df.loc[df['PHENOM'] == 'FL']
#df=df.loc[df['PHENOM'] == 'FZ']
df=df.loc[df['PHENOM'] == 'FA']

df['num_days'] = (pd.to_datetime(df['EXPIRED']) - pd.to_datetime((df['ISSUED']))).dt.days + 1

example = df.groupby(pd.to_datetime(df['ISSUED']).dt.year)
final_result = example.count()
final_result['TOTAL_DAYS'] = example.sum()['num_days']
final_result['average_duration'] = final_result['TOTAL_DAYS'] / final_result['ISSUED']
df2 = pd.read_csv('/content/drive/MyDrive/WEATHER/MASTER-data-table-2014-2022_cleaned.csv', usecols = ['YEAR', 'STATE', 'HOMICIDE RATE', 'DEATHS'])
#change state name to match file
df2=df2.loc[df2['STATE'] == 'SD']
final_result.rename(columns={ final_result.columns[0]: "count_ISSUED" }, inplace = True)

#COVID
#df2.drop(df2[df2['YEAR'] == '2020'].index)
#final_result.drop(final_result[final_result['ISSUED'] == '2020'].index)

f = final_result.join(df2.set_index('YEAR'), on='ISSUED')


In [226]:
f['HOMICIDE RATE'].corr(f['TOTAL_DAYS'])

-0.4678109607369383

In [227]:
f['HOMICIDE RATE'].corr(f['count_ISSUED'])

-0.4504349913247523

In [228]:
f['HOMICIDE RATE'].corr(f['average_duration'])

-0.6981267076100999

In [229]:
from scipy.stats import pearsonr


In [230]:
pearsonr(f['HOMICIDE RATE'], f['count_ISSUED'])

PearsonRResult(statistic=-0.4504349913247524, pvalue=0.2237131041967222)

In [231]:
pearsonr(f['HOMICIDE RATE'], f['TOTAL_DAYS'])

PearsonRResult(statistic=-0.4678109607369383, pvalue=0.20412698736475487)

In [232]:
pearsonr(f['HOMICIDE RATE'], f['average_duration'])

PearsonRResult(statistic=-0.6981267076100999, pvalue=0.03648300018323236)

In [233]:
print(final_result.columns.tolist())

['count_ISSUED', 'EXPIRED', 'PHENOM', 'SIG', 'num_days', 'TOTAL_DAYS', 'average_duration']


In [234]:
print(df)
print(final_result)

                 ISSUED           EXPIRED PHENOM SIG  num_days
0      2018-05-19 16:01  2018-05-19 21:45     FA   W         1
60     2018-05-30 00:15  2018-05-31 00:15     FA   W         2
62     2019-08-11 02:02  2019-08-11 08:00     FA   W         1
112    2018-04-12 21:18  2018-04-14 21:15     FA   W         2
139    2019-03-25 16:49  2019-03-27 04:45     FA   W         2
...                 ...               ...    ...  ..       ...
68767  2022-07-07 17:55  2022-07-08 05:45     FA   W         1
68978  2022-08-06 06:28  2022-08-06 15:30     FA   W         1
68979  2022-08-06 06:28  2022-08-06 15:30     FA   W         1
68980  2022-08-06 06:34  2022-08-06 12:30     FA   W         1
68981  2022-08-06 06:34  2022-08-06 12:30     FA   W         1

[972 rows x 5 columns]
        count_ISSUED  EXPIRED  PHENOM  SIG  num_days  TOTAL_DAYS  \
ISSUED                                                             
2014              69       69      69   69        69         113   
2015            

In [235]:
df = pd.read_csv('CO_wwa_201401010000_202212310000.csv',usecols=['ISSUED', 'EXPIRED', 'SIG', 'PHENOM'])
df=df.loc[df['SIG'] == 'W']
#change PHENOM for each Pheno. Comment out below line if finding total corelation
df=df.loc[df['PHENOM'] == 'EH']
#df=df.loc[df['PHENOM'] == 'FF']
#df=df.loc[df['PHENOM'] == 'TO']
#df=df.loc[df['PHENOM'] == 'SV']
df['num_days'] = (pd.to_datetime(df['EXPIRED']) - pd.to_datetime((df['ISSUED']))).dt.days

df2 = len(df['num_days'] < -1 )
print("Num of rows where num_days < 0 is ", df2)
print(df)
df.to_csv('test.csv')

Num of rows where num_days < 0 is  2
                 ISSUED           EXPIRED PHENOM SIG  num_days
49735  2021-06-14 18:00  2021-06-18 22:10     EH   W         4
49836  2021-06-14 18:00  2021-06-18 22:10     EH   W         4
